In [16]:
%run w2d1.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from collections import OrderedDict
from typing import Callable, Dict, Optional, List

import torch as t
from torch import nn
import torch.nn.functional as F
from torch import einsum
from einops import rearrange, reduce, repeat
import bert_tests
import matplotlib.pyplot as plt

In [19]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
print(tokenizer("hello what's up"))
uncased_tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
print(uncased_tokenizer(["hello what's up"]))
coded = uncased_tokenizer(["hello what's up"])
uncased_tokenizer.batch_decode(coded['input_ids'])
tokenizer.batch_decode(coded['input_ids'])
uncased_tokenizer.batch_decode(coded['input_ids'])


{'input_ids': [101, 19082, 1184, 112, 188, 1146, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[101, 7592, 2054, 1005, 1055, 2039, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


["[CLS] hello what's up [SEP]"]

In [22]:
my_bert, pretrained_bert = load_pretrained_bert()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
def feed_bert(model: nn.Module, text: str, tokenizer, top_k: int = 10):

    input_ids: List[int] = tokenizer(text)["input_ids"]
    print(input_ids)
    mask_idxs = [idx for idx, token in enumerate(input_ids) if token == 103]

    print(mask_idxs)

    all_logits = model(t.tensor([input_ids], dtype=t.long))[0]

    for mask_idx in mask_idxs:
        logits = all_logits[mask_idx]
        print(logits.shape)
        probs = t.softmax(logits, dim=0)

        top_logit_idxs = t.argsort(logits, descending=True)[:top_k]
        print(top_logit_idxs)
        
        top_logit_words = tokenizer.decode(top_logit_idxs)
        print(top_logit_words)


feed_bert(my_bert, "The fish loves to eat [MASK].", tokenizer, top_k=20)


[101, 1109, 3489, 7871, 1106, 3940, 103, 119, 102]
[7]
torch.Size([28996])
tensor([ 119,  132,  106, 1232,  136,  117,  131,  107,  197,  112,  118,  635,
         114, 1116,  100, 1105,  113, 1272, 1183, 1112])
. ;!...?, : " |'- । )s [UNK] and ( becausey as
